In [1]:
%load_ext autoreload
%autoreload 2
%cd "python/LatentEvolution"
%ls

/home/hew/python/LatentEvolution
cache/  data/  env.txt  figure/  framework/  main.py  script/  temp/


In [2]:
import esm
import numpy as np
import pandas as pd
import torch
from scipy.stats import pearsonr, spearmanr

from framework.utils.lightning.trainer_utils import LitInference
from script.task_02_ProteinVAE.ProteinVAE.sequence_data_module import SequenceDataModule
from script.task_02_ProteinVAE.ProteinVAE.sequence_lightning_module import SequenceLightningModule

========== add root_path and framework_path to sys.path ==========
root_path: /home/hew/python/LatentEvolution
framework_path: /home/hew/python/LatentEvolution/framework


In [3]:
torch.__version__, esm.__version__

('2.1.1', '2.0.0')

In [4]:
version = 0
epoch = '88, loss=35.912, ce=18.633, reg=10.772, mse=6.506, ddG=0.531, dS=0.832.ckpt'
log_dir = "./script/task_02_ProteinVAE/lightning_logs/"
inferencer = LitInference(SequenceLightningModule, SequenceDataModule, log_dir, version, epoch)
predictions = inferencer.predict('test')
predictions

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42


============================== PeptideDataModule __init__ Start ==============================
============================== PeptideDataModule __init__ End ==============================
============================== prepare_data: mode[predict] Start ==============================
prepare_data predict_data kwargs {'predict_data': 'test'}
============================== prepare_data: mode[test] Start ==============================
============================== prepare_dataset Start ==============================
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< load data according to selected index >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


  0%|          | 0/2404 [00:00<?, ?it/s]

the dataset has not been partitioned, split dataset with specific ratio


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


dataframe partition values:
partition
train    1661
test      521
valid     222
Name: count, dtype: int64
select the subset for debug, max_len: 83, ratio: 1, number: 2404
============================== prepare_dataset End ==============================
len(self.test_dataset) 521
============================== prepare_data mode[test] End ==============================
len(self.predict_index) 521
============================== prepare_data mode[predict] End ==============================
============================== Setup [test] Start ==============================
[len self.predict_dataset] 521
============================== Setup [test] End ==============================


Predicting: 0it [00:00, ?it/s]

[{'recon_tokens': tensor([[ 8, 11, 12,  ...,  5,  4, 16],
          [ 6, 11, 12,  ...,  5,  4, 16],
          [ 8,  8, 12,  ...,  5,  4, 16],
          ...,
          [ 8, 11, 12,  ...,  5,  4, 16],
          [ 8, 11, 12,  ...,  5,  4, 16],
          [ 8, 11, 12,  ..., 16,  4, 16]]),
  'tokens': tensor([[ 8, 11, 12,  ...,  5,  4, 16],
          [ 5, 11, 12,  ...,  5,  4, 16],
          [ 8,  8, 12,  ...,  5,  4, 16],
          ...,
          [ 8, 11, 12,  ...,  5,  4, 16],
          [ 8, 11, 12,  ...,  5,  4, 16],
          [ 8, 11, 12,  ...,  5,  4, 16]]),
  'ddG': tensor([-0.7306, -0.7681, -1.3282, -0.0167, -1.2203, -0.7663, -1.8396, -0.1471,
          -0.1454, -0.8561, -1.4395, -1.9383, -0.5685, -0.4398, -1.1382, -4.4631,
          -1.3262, -0.7398, -1.0461, -1.7806, -0.8554, -1.4865, -1.5717, -0.1668,
          -1.2113, -0.0279, -1.8086, -1.5696, -1.8183, -1.7443, -0.0988, -0.2069,
          -0.2052, -2.4691, -4.8645, -1.2536, -1.0251, -0.9982, -1.8109, -0.2505,
          -0.1961, 

In [5]:
true_tokens = torch.concat([batch['tokens'] for batch in predictions])
recon_tokens = torch.concat([batch['recon_tokens'] for batch in predictions])
predict_z = torch.concat([batch['z'] for batch in predictions])
predict_mean = torch.concat([batch['mean'] for batch in predictions])
predict_logvar = torch.concat([batch['logvar'] for batch in predictions])
predict_ddG = torch.concat([batch['pred_ddG'] for batch in predictions])
predict_dS = torch.concat([batch['pred_dS'] for batch in predictions])
true_ddG = torch.concat([batch['ddG'] for batch in predictions])
true_dS = torch.concat([batch['dS'] for batch in predictions])

In [6]:
pearsonr(predict_ddG, true_ddG)[0], pearsonr(predict_dS, true_dS)[0]

(0.5067621742678847, 0.8133953925645634)

## load train data

In [7]:
version = 0
diffusion_model_train_data = f'./script/task_04_LatentRegression/version_{version}_train_data.pt'
train_data = torch.load(diffusion_model_train_data)
embeddings = train_data['z']
ddG = train_data['ddG']
dS = train_data['dS']

FileNotFoundError: [Errno 2] No such file or directory: './script/task_04_LatentRegression/version_3_train_data.pt'

In [ ]:
min(ddG), max(dS)

## model predict

In [7]:
log_dir = "./script/task_02_ProteinVAE/lightning_logs/"
# version = 2
# epoch = '95, loss=35.447, ce=18.697, reg=10.702, mse=6.049, ddG=0.537, dS=0.861.ckpt'

version = 3
epoch = '95, loss=35.447, ce=18.697, reg=10.702, mse=6.049, ddG=0.537, dS=0.861.ckpt'
inferencer = LitInference(SequenceLightningModule, SequenceDataModule, log_dir, version, epoch)
tokenizer = inferencer.pl_data_module.tokenizer
device = torch.device('cuda:0')
# model = inferencer.model
# model = model.eval()
# model

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 42


============================== PeptideDataModule __init__ Start ==============================
============================== PeptideDataModule __init__ End ==============================


In [15]:
# input latent vectors, output generated sequences
def decode_z(z):
    logits = model.decode(z)[:, 1:-1]
    tokens = torch.argmax(logits, dim=-1)
    sequences = tokenizer.decode(tokens)
    return sequences


# input amino acid sequences, output attributes
def model_forward(seqs):
    # inferencer.set_batch_size(128)
    predictions = inferencer.predict(seqs)
    # true_tokens = torch.concat([batch['tokens'] for batch in predictions])
    # recon_tokens = torch.concat([batch['recon_tokens'] for batch in predictions])
    # predict_z = torch.concat([batch['z'] for batch in predictions])
    # predict_mean = torch.concat([batch['mean'] for batch in predictions])
    # predict_logvar = torch.concat([batch['logvar'] for batch in predictions])
    predict_ddG = torch.concat([batch['pred_ddG'] for batch in predictions])
    predict_dS = torch.concat([batch['pred_dS'] for batch in predictions])
    true_ddG = torch.concat([batch['ddG'] for batch in predictions])
    true_dS = torch.concat([batch['dS'] for batch in predictions])
    return predict_ddG, predict_dS, true_ddG, true_dS

In [16]:
z = embeddings[:2].to(device)
gen_seqs = decode_z(z)
gen_seqs

NameError: name 'model' is not defined

In [18]:
seqs = [
    'STIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDQWSAFLKEQSTIAQMYPLQEIKNLTVKLVLQALQ',
    'STEEEQAKTFLDKFNHEAEDYFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLRALL',
    'STIEEQAKIFLDKFNHEAEDLFYQSSLRSFNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLQALQ'
]
predict_ddG, predict_dS, true_ddG, true_dS = model_forward(seqs)
predict_ddG, predict_dS, true_ddG, true_dS

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


============================== prepare_data: mode[predict] Start ==============================
prepare_data predict_data kwargs {'predict_data': ['STIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDQWSAFLKEQSTIAQMYPLQEIKNLTVKLVLQALQ', 'STEEEQAKTFLDKFNHEAEDYFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLRALL', 'STIEEQAKIFLDKFNHEAEDLFYQSSLRSFNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLQALQ']}
len(self.predict_index) 3
============================== prepare_data mode[predict] End ==============================
============================== Setup [test] Start ==============================
[len self.predict_dataset] 3
============================== Setup [test] End ==============================


Predicting: |          | 0/? [00:00<?, ?it/s]

(tensor([0.0345, 0.0346, 0.0427]),
 tensor([-0.0618, -0.0664, -0.0630]),
 tensor([nan, nan, nan]),
 tensor([nan, nan, nan]))

In [1]:
seqs = [
    'STIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDQWSAFLKEQSTIAQMYPLQEIKNLTVKLVLQALQ',
    'STEEEQAKTFLDKFNHEAEDYFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLRALL',
    'STIEEQAKIFLDKFNHEAEDLFYQSSLRSFNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLQALQ'
]
predict_ddG, predict_dS, true_ddG, true_dS = model_forward(seqs)
predict_ddG, predict_dS, true_ddG, true_dS

NameError: name 'model_forward' is not defined

In [10]:
df = pd.read_csv('./data/ACE2_variants_2k/cooked/ACE2_variants_2k.csv')
df

,index,name,partition,length,sequence,structure,graph,ddG,dS
0,0,0,NaN,83,STIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNN...,NaN,NaN,-1.0838,0.017
1,1,1,NaN,83,STIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNN...,NaN,NaN,-0.0154,0.017
2,10,10,NaN,83,STIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNN...,NaN,NaN,-0.8987,0.017
3,100,100,NaN,83,STIEEQAKTFLDKFNHDAEDLFYQSFLASWNYNTNITEENVQNMNN...,NaN,NaN,-1.1936,0.017
4,1000,1000,NaN,83,SDIEEQAKTFLDKFNHEAEDLFYQSSLAYWNYNTNITEENVQNMGN...,NaN,NaN,-2.5357,0.017
...,...,...,...,...,...,...,...,...,...
2399,995,995,NaN,83,STIEEQAKTFLDKFNHEAEDLFYQSDLARWNYNTNITEENVQNMNN...,NaN,NaN,-0.4275,0.017
2400,996,996,NaN,83,STIEEQAKTFLDKFNHEAEDLFYQSSLASWWYNTNITEENVQNMNN...,NaN,NaN,-1.6738,0.017
2401,997,997,NaN,83,STIEEQAKTFLDKFNHEAEDLFYQMSLASWNYNTNITEENVQNMNN...,NaN,NaN,-1.9267,0.017
2402,998,998,NaN,83,SDIEEQAKMFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNN...,NaN,NaN,-1.3416,0.017


In [11]:
seqs = df.sequence.tolist()
true_ddG = df.ddG.tolist()
true_dS = df.dS.tolist()
predict_ddG, predict_dS, _, _ = model_forward(seqs)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


============================== prepare_data: mode[predict] Start ==============================
prepare_data predict_data kwargs {'predict_data': ['...']}
len(self.predict_index) 2404
============================== prepare_data mode[predict] End ==============================
============================== Setup [test] Start ==============================
[len self.predict_dataset] 2404
============================== Setup [test] End ==============================


Predicting: 0it [00:00, ?it/s]

In [12]:
predict_ddG

tensor([0.7948, 0.9767, 0.7347,  ..., 0.6900, 0.5558, 0.4590])

In [13]:
pearsonr(predict_ddG, true_ddG)[0], pearsonr(predict_dS, true_dS)[0]

(0.8245244368982572, 0.8985297411531991)

In [14]:
spearmanr(predict_ddG, true_ddG)[0], spearmanr(predict_dS, true_dS)[0]

(0.7472712859713769, 0.5157004220257524)

In [15]:
sorted_true_ddG_indices = np.argsort(true_ddG)
sorted_true_ddG = np.array(true_ddG)[sorted_true_ddG_indices]
corresponding_predict_ddG = np.array(predict_ddG)[sorted_true_ddG_indices]
np.concatenate([sorted_true_ddG.reshape(-1, 1), corresponding_predict_ddG.reshape(-1, 1)], axis=1)

array([[-5.54970000e+00, -8.95914555e-01],
       [-5.37360000e+00, -7.41741717e-01],
       [-4.86450000e+00, -1.74640000e-01],
       ...,
       [-4.00000000e-03,  9.90709305e-01],
       [-2.60000000e-03,  9.69277203e-01],
       [-7.00000000e-04,  9.33099389e-01]])

In [16]:
sorted_true_dS_indices = np.argsort(true_dS)[::-1]
sorted_true_dS = np.array(true_dS)[sorted_true_dS_indices]
corresponding_predict_dS = np.array(predict_dS)[sorted_true_dS_indices]
np.concatenate([sorted_true_dS.reshape(-1, 1), corresponding_predict_dS.reshape(-1, 1)], axis=1)

array([[ 0.078     ,  0.83474034],
       [ 0.078     ,  0.93038553],
       [ 0.078     ,  0.93465209],
       ...,
       [ 0.005     , -0.97581601],
       [ 0.005     , -0.94977134],
       [ 0.005     , -0.97757703]])

## train

## generation

min ddG: -6.0, lower
max dS: 0.086, higher


1. train diffusion model
2. modify ddG, dS, gen z
3. model.decode(z) -> GenSet (100000)
4. model(GenSet) -> pred_ddG, pred_dS
5. rank GenSet by pred_ddG, pred_dS
6. select top 1000
7. evaluate using FoldX. Protein-Sol